# Day 8

## Part 1

In [1]:
f = open("./data/input8.txt", "r")
c = f.readlines()


In [2]:
import pandas as pd
df = pd.DataFrame(c)
    

In [19]:
tmp = df[0].str.split('|').apply(pd.Series)
input = tmp[0]
output = tmp[1]

In [20]:
input = input.str.split(' ').apply(pd.Series).drop(10, axis=1)
output = output.str.split(' ').apply(pd.Series).drop(0, axis=1)
output[4] = output[4].str.replace('\n','')

In [30]:
res = []
for col in input.columns:
    res.append(input[col].str.len())
lengths = pd.concat(res, axis=1)

In [31]:
4 - has length 4
1 - has length 2
7 - has length 3
8 - has length 7

,0,1,2,3,4,5,6,7,8,9
0,6,6,4,5,6,2,5,5,7,3
1,7,3,5,6,6,6,5,5,4,2
2,7,4,6,3,6,5,5,5,2,6
3,4,6,5,6,2,7,6,5,3,5
4,6,5,2,5,7,6,3,6,5,4
...,...,...,...,...,...,...,...,...,...,...
195,2,6,5,7,3,6,5,6,5,4
196,5,4,6,6,5,7,3,6,5,2
197,5,3,6,6,6,4,5,5,2,7
198,6,5,7,6,4,3,6,2,5,5


In [51]:
res = []
for col in output.columns:
    res.append(output[col].str.len())
output_lengths = pd.concat(res, axis=1)

In [53]:
(output_lengths==4).sum().sum() + (output_lengths==2).sum().sum() +  (output_lengths==3).sum().sum() +  (output_lengths==7).sum().sum()

303

## Part 2

In [98]:
def get_digit(dict, str):
   
    if len(str) == 5:
        if len(set(dict[1])&set(str))==2:
            dict[3] = str
        elif len(set(dict[4])& set(str))==3:
            dict[5] = str
        else:
            dict[2]=str
        return dict
    elif len(str) == 6:
        if len(set(dict[7])& set(str))==2:
            dict[6] = str
        elif len(set(dict[4])&set(str))==4:
            dict[9]=str
        else:
            dict[0]=str
        return dict
    else:
        return dict

        
        

In [106]:
def find_digits(vv):
    dict = {}
    dict[1] = [x for x in vv if len(x)==2][0]
    dict[4] = [x for x in vv if len(x)==4][0]
    dict[7] = [x for x in vv if len(x)==3][0]
    dict[8] = [x for x in vv if len(x)==7][0]

    for v in vv:
        digits = get_digit(dict, v)
    return digits

digit_finder = lambda x: int(''.join([str(inv_dig[''.join(sorted(y))]) for y in x]))


In [111]:
import numpy as np
final_arr = []
for i in np.arange(input.shape[0]):
    digits = find_digits(input.loc[i].to_list())
    inv_dig = {''.join(sorted(v)): k for k, v in digits.items()}
    number = digit_finder(output.loc[i].to_list())
    final_arr.append(number)

In [110]:
np.array(final_arr).sum()


961734

# Day 9

## Part 1

In [1]:
import pandas as pd
import numpy as np

In [2]:
f = open("./data/input9.txt", "r")
c = f.readlines()
df = pd.DataFrame(c)
df = df[0].apply(lambda x: list(x)).apply(pd.Series).drop(100, axis=1)
df = df.astype(int)
df_t = df.T



In [3]:
a = df < df.shift(1).fillna(10)
b = df < df.shift(-1).fillna(10)
c = (df_t < df_t.shift(1).fillna(10)).T
d = (df_t < df_t.shift(-1).fillna(10)).T


In [4]:
df[(a==True)&(b==True)&(c==True)&(d==True)].sum().sum(), df[(a==True)&(b==True)&(c==True)&(d==True)].count().sum()


(276.0, 224)

In [5]:
276+224

500

## Part 2

In [6]:
arr = df[(a==True)&(b==True)&(c==True)&(d==True)].fillna(999).to_numpy()
low_points = np.where(arr!=999)

In [7]:
x = low_points[0]
y = low_points[1]
low_points = pd.DataFrame(zip(x,y))

In [143]:
low_points.shape

(224, 2)

In [179]:
def discover_basin(basin, df, x, y):
    
    # search right
    if df.loc[x,y]==9:
        return basin
    
    if not (x,y) in basin:
        if x>0:
            basin.append((x,y))
            discover_basin(basin, df, x-1, y)
        if x < df.shape[0]-1:
            
            basin.append((x,y))
            discover_basin(basin, df, x+1, y)
        if y>0:
            
            basin.append((x,y))
            discover_basin(basin, df, x, y-1)
        if y < df.shape[1]-1:
            
            basin.append((x,y))
            discover_basin(basin, df, x, y+1)
    return basin    
    
    

In [184]:
%%timeit
all_basins = []

for _, row in low_points.iterrows():
    basin = []
    res = discover_basin(basin, df, row[0], row[1])
    all_basins.append(len(list(set(res))))

272 ms ± 3.53 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [181]:
pd.Series(all_basins).sort_values(ascending=False).head(3)

132    100
96      99
41      98
dtype: int64

In [176]:
100*99*98

970200

# Day 10 

## Part 1

In [252]:
f = open("./data/input10.txt", "r")
c = f.readlines()
df = pd.DataFrame(c)
df[0] = df[0].str.replace('\n','')

In [285]:
opening_chars = ["(","<","{","["]
open_close_dict = {"(":")","{":"}","[":"]","<":">"}
error_points = {")":3,"]":57,"}":1197,">":25137}
closing_points = {")":1,"]":2,"}":3,">":4}

In [267]:
def find_first_bad_closing_only_dict_and_queus_no_pandas(sequence):
    # print(sequence)
    q = []
    for char in sequence:
        if char in opening_chars:
            q.append(char)
        else:
            last  = q.pop(-1)
            if open_close_dict[last]!=char:
                return char

            

In [268]:
arr = []
for _, row in df.iterrows():
    row = row.values[0]
    arr.append(find_first_bad_closing_only_dict_and_queus_no_pandas(row))

mapped_arr = [error_points[x] if x is not None else 0 for x in arr ]
np.array(mapped_arr).sum()

392421

## Part 2

In [277]:
incomplete = np.where(np.array(mapped_arr)==0)
print(len(incomplete[0]))
incomplete_df = df.loc[incomplete]
print(incomplete_df.shape)

55
(55, 1)


In [278]:
def find_open_seqs(sequence):
    # print(sequence)
    q = []
    for char in sequence:
        if char in opening_chars:
            q.append(char)
        else:
            last  = q.pop(-1)
    return q


def fill_in_q(q):
    new_q = []
    while len(q)>0:
        open = q.pop(-1)
        new_q.append(open_close_dict[open])
    return new_q
        

In [279]:
final_result = []
for _, row in incomplete_df.iterrows():
    row = row.values[0]
    open_q = find_open_seqs(row)
    final_result.append(fill_in_q(open_q))

In [286]:
def calculate_sequence_score(seq):
    score = 0
    for char in seq:
        score = score * 5
        score += closing_points[char]
    return score
    

In [287]:
all_scores = [calculate_sequence_score(x) for x in final_result]

In [290]:
sorted(all_scores)[27]

2769449099

# Day 11

## Part 1

In [428]:
f = open("./data/input11.txt", "r")
c = f.readlines()
c = [x.replace("\n","") for x in c]
c = [list(x) for x in c]
df = pd.DataFrame(c).astype("float64")


In [430]:
def increase_neighbors(arr , x):
    mod_res = np.mod(x,10)
    neighbors = [x+1, x+9, x+10, x+11, x-1, x-11, x-10, x-9]
    # print(neighbors, '\n')
    legal_neighbors = [y for y in neighbors if (y>0) & (np.mod(y,10) in [mod_res, mod_res-1, mod_res+1])& (y<100)]
    arr[legal_neighbors] +=1
    return arr
    

In [431]:
arr = df.to_numpy().flatten()
flashes = 0
for i in np.arange(100):
    print('step', i)
    arr += 1
    while len(np.where(arr>9)[0]) > 0:
        flash_candidate = np.where(arr>9)
        flashes += len(flash_candidate[0])
        print(flashes)
        for x in flash_candidate[0]:
            arr = increase_neighbors(arr , x)
            arr[flash_candidate[0]]= np.nan
        arr[np.where(arr == np.nan)] = 0




step 0
step 1
step 2
step 3
step 4
step 5
step 6
step 7
step 8
step 9
step 10
step 11
step 12
step 13
step 14
step 15
step 16
step 17
step 18
step 19
step 20
step 21
step 22
step 23
step 24
step 25
step 26
step 27
step 28
step 29
step 30
step 31
step 32
step 33
step 34
step 35
step 36
step 37
step 38
step 39
step 40
step 41
step 42
step 43
step 44
step 45
step 46
step 47
step 48
step 49
step 50
step 51
step 52
step 53
step 54
step 55
step 56
step 57
step 58
step 59
step 60
step 61
step 62
step 63
step 64
step 65
step 66
step 67
step 68
step 69
step 70
step 71
step 72
step 73
step 74
step 75
step 76
step 77
step 78
step 79
step 80
step 81
step 82
step 83
step 84
step 85
step 86
step 87
step 88
step 89
step 90
step 91
step 92
step 93
step 94
step 95
step 96
step 97
step 98
step 99


In [432]:
flashes

100

In [337]:
df

,0,1,2,3,4,5,6,7,8,9
0,7,3,1,3,5,1,1,5,5,1
1,3,7,2,4,8,5,5,8,6,7
2,2,3,7,4,3,3,1,5,7,1
3,4,4,3,8,2,1,3,4,3,7
4,6,5,1,1,5,6,6,2,8,7
5,6,7,2,7,2,4,5,5,3,2
6,3,7,3,6,8,6,8,6,6,2
7,2,3,4,8,1,3,8,2,6,3
8,2,4,1,7,4,8,3,1,2,1
9,8,8,1,2,6,1,7,1,1,2


In [329]:
import numpy as np
arr = df.to_numpy()
np.lib.stride_tricks.sliding_window_view(arr, (3,3))


array([[[[7, 3, 1],
         [3, 7, 2],
         [2, 3, 7]],

        [[3, 1, 3],
         [7, 2, 4],
         [3, 7, 4]],

        [[1, 3, 5],
         [2, 4, 8],
         [7, 4, 3]],

        [[3, 5, 1],
         [4, 8, 5],
         [4, 3, 3]],

        [[5, 1, 1],
         [8, 5, 5],
         [3, 3, 1]],

        [[1, 1, 5],
         [5, 5, 8],
         [3, 1, 5]],

        [[1, 5, 5],
         [5, 8, 6],
         [1, 5, 7]],

        [[5, 5, 1],
         [8, 6, 7],
         [5, 7, 1]]],


       [[[3, 7, 2],
         [2, 3, 7],
         [4, 4, 3]],

        [[7, 2, 4],
         [3, 7, 4],
         [4, 3, 8]],

        [[2, 4, 8],
         [7, 4, 3],
         [3, 8, 2]],

        [[4, 8, 5],
         [4, 3, 3],
         [8, 2, 1]],

        [[8, 5, 5],
         [3, 3, 1],
         [2, 1, 3]],

        [[5, 5, 8],
         [3, 1, 5],
         [1, 3, 4]],

        [[5, 8, 6],
         [1, 5, 7],
         [3, 4, 3]],

        [[8, 6, 7],
         [5, 7, 1],
         [4, 3, 7]]],


    